In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.decomposition import PCA
import pandas.util.testing as tm
import pickle

In [9]:
def combine_rows(df):
    cols = df.columns
    df = df.groupby(df.index//NUM_OF_RECORDS_PER_SAMPLE)[cols].apply(lambda x: pd.DataFrame(x.values)).unstack().reset_index()
    df.columns = df.columns.droplevel()
    new_cols = ["index"]
    for col in cols:
        for i in range(NUM_OF_RECORDS_PER_SAMPLE):
            new_cols.append(col+str(i)) 
        
    df.columns = new_cols
    not_important_features = ["index"]
    features = ["Time","Frame ID","state_type","name","Hand Type","# hands"]
    for f in features:
        for s in new_cols:
            if f in s:
                df[f] = df[s]
                not_important_features.append(s)
    df = df.drop(not_important_features,axis=1)
    numeric_featers = [x for x in df.columns if x not in features]
    df[numeric_featers] = df[numeric_featers].apply(pd.to_numeric)
    return df

In [10]:
def get_state_type(filename):
    state_type = 3
    if 'Spontan' in filename:
        state_type = 1
    elif 'Sync' in filename:
        state_type = 2
    return state_type

def handle_right_hand_file(df):
    alone_right = pd.read_csv("HandRight.csv",index_col=None, header=0)
    alone_right.rename(columns=lambda x: x.strip(), inplace=True)
    alone_right["state_type"] = 3
    alone_right["name"] = ""
    return df

In [12]:
def combine_datasets(path):
    all_files = glob.glob(path)
    lst = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df.rename(columns=lambda x: x.strip(), inplace=True)        
        df['state'] = define_state(filename)
        start_time = df["Time"].min()
        df = df[df["Time"]>start_time+7]
        df = combine_rows(df)
        df["Time"] = df["Time"].astype("float")
        df = df[df['state'].notna()]
        df["state"] = df["state"].astype("int")
        lst.append(df)   
    df = pd.concat(lst, axis=0, ignore_index=True)
    df = hand_right_file(df)
    return df